In [1]:
#필수 라이브러리
import sys
import csv
import pandas as pd
import numpy as np
import re
import time

In [1]:
#워드 클라우드
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#konlpy 라이브러리
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from eunjeon import Mecab

In [3]:
#pdf저장 라이브러리
import glob
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from PIL import Image
import cv2

In [4]:
#df-idf 정의
# docs = okja

# N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def tf_log(t, d):
    return log(d.count(t)+1)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [55]:
#특수문자 제거 정의
def clean_signal(text):
    cleaned = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", str(text))
    return cleaned

def clean_en(text):
    cleaned = re.sub(r"[^\uAC00-\uD7A30-9\s]", "", str(text))
    return cleaned

In [4]:
hannanum = Hannanum()
okt = Okt()
komoran = Komoran(max_heap_size= 1024 * 6)
kkma = Kkma()
mecab = Mecab()

In [9]:
# #csv
# f = open("D:\crawl_new_crime\네이버뉴스_신종범죄_2011.csv",newline = '',encoding='utf-8')
# rdr = csv.reader(f)
# print(rdr)

# csv_list = []

# for line in rdr:
#     csv_list.append(line[1])

# # csv_list

In [5]:
#excel

current_path = os.getcwd()

f = pd.read_excel("./data/공고목록.xls")

csv_list = f['공고명'].values.tolist()

csv_list

In [16]:
# docs = csv_list

# print(docs[0])
# if (docs[0] == "title"):
#     del docs[0]
# print(docs[0])

In [17]:
#user parameter

#형태소 갯수
num = 5

#동사원형으로 바꾸는 코드의 갯수
num_okt = 4

#레이블 숫자
label = 0

#저장위치
output_file = "result"

#기사 이름
name = "practice"

In [18]:
#기본 = 0, 확장 = 1
expand = input("expand = ")
# print(type(expand))
if expand == "0":
    print("옵션 2개")
elif expand == "1":
    print("옵션 4개")

expand = 0
옵션 2개


In [19]:
#동사원형 X = 3, 동사원형 O = 4
verb_form = input("verb_form = ")
# print(type(verb_form))
if verb_form == "3":
    print("동사원형 X")
elif verb_form == "4":
    print("동사원형 O")

verb_form = 3
동사원형 X


In [21]:
# 0: hannanum, 1: okt, 2: kkma, 3: komoran, 4: mecab
# 방법 정하기

methods = 0

In [22]:
#형태소 분석 pos추출(hannanum, okt, kkma, komoran, mecab)


sentences_tag = []
sentences_tag_2 = []

#hannanum
if methods == 0:
    hannanum_list = []
    hannanum_byokt = []
    start_time = time.time()

    for sentence in csv_list:
        pos = hannanum.pos(sentence)
        sentences_tag.append(pos)

    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['N']:
                sentences_tag_2.append(word)
            elif tag in ['P']:
                if len(word)>1:
                    sentences_tag_2.append(word)

    for word in sentences_tag_2:
        clean_word = clean_en(word)
        hannanum_list.append(clean_word)

    end_time = time.time()    
    total_time_hannanum = end_time - start_time

    hannanum_byokt = hannanum_list
    print("hannanum시간: "+total_time_hannanum)

#okt
elif methods == 1:
    okt_list = []
    okt_byokt = []
    start_time = time.time()

    if (verb_form == "3"):
        for sentence in csv_list:
            pos = okt.pos(sentence, norm = True)
            sentences_tag.append(pos)

    elif (verb_form == "4"):
        for sentence in csv_list:
            pos = okt.pos(sentence, norm = True, stem =True)
            sentences_tag.append(pos)

    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun']:
                sentences_tag_2.append(word)
            elif tag in ['Verb']:
                if len(word)>1:
                    sentences_tag_2.append(word)

    for word in sentences_tag_2:
        clean_word = clean_en(word)
        okt_list.append(clean_word)

    end_time = time.time()    
    total_time_okt = end_time - start_time

    okt_byokt = okt_list
    print("okt시간: "+total_time_okt)

#kkma
elif methods == 2:
    kkma_list = []
    kkma_byokt = []
    start_time = time.time()

    for sentence in csv_list:
        pos = kkma.pos(sentence)
        sentences_tag.append(pos)

    #오리지널
    if expand == "0":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    #확장
    elif expand == "1":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)
                elif tag in ['NNP']:
                    sentences_tag_2.append(word)
                elif tag in ['NP']:
                    sentences_tag_2.append(word)
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    for word in sentences_tag_2:
        clean_word = clean_en(word)
        kkma_list.append(clean_word)

    end_time = time.time()
    total_time_kkma = end_time - start_time

    kkma_byokt = kkma_list
    print("kkma시간: "+total_time_kkma)

#komoran
elif methods == 3:
    komoran_list = []
    komoran_byokt = []
    csv_list2 = []

    for sentence in csv_list:
        sentence = sentence.replace('\n','').replace('\t','').replace('\r','')
        csv_list2.append(sentence)

    start_time = time.time()

    for sentence in csv_list2:
        pos = komoran.pos(sentence)
        sentences_tag.append(pos)

    #오리지널
    if expand == "0":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    #확장
    elif expand == "1":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)
                elif tag in ['NNP']:
                    sentences_tag_2.append(word)
                elif tag in ['NP']:
                    sentences_tag_2.append(word)
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    for word in sentences_tag_2:
        clean_word = clean_en(word)
        komoran_list.append(clean_word)  

    end_time = time.time()
    total_time_komoran = end_time - start_time

    komoran_byokt = komoran_list
    print("komoran시간: "+total_time_komoran)

#mecab
elif methods == 4:
    mecab_list = []
    mecab_byokt = []
    start_time = time.time()

    for sentence in csv_list:
        pos = mecab.pos(sentence)
        sentences_tag.append(pos)

    #오리지널
    if expand == "0":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)        
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    #확장
    elif expand == "1":
        for sentence1 in sentences_tag:
            for word, tag in sentence1:
                if tag in ['NNG']:
                    sentences_tag_2.append(word)
                elif tag in ['NNP']:
                    sentences_tag_2.append(word)
                elif tag in ['NP']:
                    sentences_tag_2.append(word)
                elif tag in ['VV']:
                    if len(word)>1:
                        sentences_tag_2.append(word)

    for word in sentences_tag_2:
        clean_word = clean_en(word)
        mecab_list.append(clean_word)

    end_time = time.time()    
    total_time_mecab = end_time - start_time

    mecab_byokt = mecab_list
    print("mecab시간: "+total_time_mecab)


13.05027723312378


In [23]:
if verb_form == "3":
    print("동사원형 X")
elif verb_form == "4":
    print("동사원형 O")

동사원형 X


In [24]:
#hannanum, kkma, komoran, mecab 동사원형으로 바꾸기
if verb_form == "4":  
    for i in range(0, num_okt):
        start_time2 = time.time()
        sentences_tag_3 = []

        #hannanum
        if i == 0:
            hannanum_byokt = []

            for sentence in hannanum_list:
                pos = okt.pos(sentence, norm = True, stem =True)
                sentences_tag_3.append(pos)


            for sentence1 in sentences_tag_3:
                for word, tag in sentence1:
                    if tag in ['Noun']:
                        hannanum_byokt.append(word)
                    elif tag in ['Verb']:
                        if len(word)>1:
                            hannanum_byokt.append(word) 

            end_time2 = time.time()
            total_time2_hannanum = end_time2 - start_time2

            print(len(hannanum_byokt))

        #kkma
        elif i == 1:
            kkma_byokt = []

            for sentence in kkma_list:
                pos = okt.pos(sentence, norm = True, stem =True)
                sentences_tag_3.append(pos)


            for sentence1 in sentences_tag_3:
                for word, tag in sentence1:
                    if tag in ['Noun']:
                        kkma_byokt.append(word)
                    elif tag in ['Verb']:
                        if len(word)>1:
                            kkma_byokt.append(word) 

            end_time2 = time.time()
            total_time2_kkma = end_time2 - start_time2

            print(len(kkma_byokt))

        #komoran
        elif i == 2:
            komoran_byokt = []

            for sentence in komoran_list:
                pos = okt.pos(sentence, norm = True, stem =True)
                sentences_tag_3.append(pos)

            for sentence1 in sentences_tag_3:
                for word, tag in sentence1:
                    if tag in ['Noun']:
                        komoran_byokt.append(word)
                    elif tag in ['Verb']:
                        if len(word)>1:
                            komoran_byokt.append(word) 

            end_time2 = time.time()
            total_time2_komoran = end_time2 - start_time2

            print(len(komoran_byokt))

        #mecab
        elif i == 3:
            mecab_byokt = []

            for sentence in mecab_list:
                pos = okt.pos(sentence, norm = True, stem =True)
                sentences_tag_3.append(pos)


            for sentence1 in sentences_tag_3:
                for word, tag in sentence1:
                    if tag in ['Noun']:
                        mecab_byokt.append(word)
                    elif tag in ['Verb']:
                        if len(word)>1:
                            mecab_byokt.append(word) 

            end_time2 = time.time()
            total_time2_mecab = end_time2 - start_time2

            print(len(mecab_byokt))

In [25]:
#공백, 한글자, 중복제거
pos_list2 = ['폰', '낮', '밤', '밖', '돈']

j = methods

#hannanum
if j == 0:
    hannanum_list2 = []
    hannanum_list_final = []

    #공백 제거
    hannanum_byokt = list(filter(bool, hannanum_byokt))

    #한글자 제거
    for word in hannanum_byokt:
        if len(word) > 1:
            hannanum_list2.append(word)
        elif len(word) < 2:
            if word in pos_list2:
                hannanum_list2.append(word)

    hannanum_list_len = len(hannanum_list2)

    #중복 제거
    for i in range(0, hannanum_list_len):
        temp = hannanum_list2[i]
        if temp not in hannanum_list_final:
            hannanum_list_final.append(temp)

    print(len(hannanum_list_final))        

#okt
elif j == 1:
    okt_list2 = []
    okt_list_final = []

    #공백 제거
    okt_byokt = list(filter(bool, okt_byokt))

    #한글자 제거
    for word in okt_byokt:
        if len(word) > 1:
            okt_list2.append(word)
        elif len(word) < 2:
            if word in pos_list2:
                okt_list2.append(word)

    okt_list_len = len(okt_list2)

    #중복 제거
    for i in range(0, okt_list_len):
        temp = okt_list2[i]
        if temp not in okt_list_final:
            okt_list_final.append(temp)

    print(len(okt_list_final))        

#kkma
elif j == 2:
    kkma_list2 = []
    kkma_list_final = []

    #공백 제거
    kkma_byokt = list(filter(bool, kkma_byokt))

    #한글자 제거
    for word in kkma_byokt:
        if len(word) > 1:
            kkma_list2.append(word)
        elif len(word) < 2:
            if word in pos_list2:
                kkma_list2.append(word)

    kkma_list_len = len(kkma_list2)

    #중복 제거
    for i in range(0, kkma_list_len):
        temp = kkma_list2[i]
        if temp not in kkma_list_final:
            kkma_list_final.append(temp)

    print(len(kkma_list_final))        

#komoran
elif j == 3:
    komoran_list2 = []
    komoran_list_final = []

    #공백 제거
    komoran_byokt = list(filter(bool, komoran_byokt))

    #한글자 제거
    for word in komoran_byokt:
        if len(word) > 1:
            komoran_list2.append(word)
        elif len(word) < 2:
            if word in pos_list2:
                komoran_list2.append(word)

    komoran_list_len = len(komoran_list2)

    #중복 제거
    for i in range(0, komoran_list_len):
        temp =komoran_list2[i]
        if temp not in komoran_list_final:
            komoran_list_final.append(temp)

    print(len(komoran_list_final))        

#mecab
elif j == 4:
    mecab_list2 = []
    mecab_list_final = []

    #공백 제거
    mecab_byokt = list(filter(bool, mecab_byokt))

    #한글자 제거
    for word in mecab_byokt:
        if len(word) > 1:
            mecab_list2.append(word)
        elif len(word) < 2:
            if word in pos_list2:
                mecab_list2.append(word)

    mecab_list_len = len(mecab_list2)

    #중복 제거
    for i in range(0, mecab_list_len):
        temp = mecab_list2[i]
        if temp not in mecab_list_final:
            mecab_list_final.append(temp)

    print(len(mecab_list_final))        


9370


In [26]:
#빈도수 테이블
docs = csv_list
N = len(docs) # 총 문서의 수

k = methods
result = []

if k == 0:
    vocab = hannanum_list_final

    for i in range(N): # 각 문서에 대해서 아래 명령을 수행
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]        
            result[-1].append(tf(t, d))

    hannanum_tf_table = pd.DataFrame(result, columns = vocab)
    hannanum_tf_table['label'] = label

elif k == 1:
    vocab = okt_list_final

    for i in range(N): # 각 문서에 대해서 아래 명령을 수행
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]        
            result[-1].append(tf(t, d))

    okt_tf_table = pd.DataFrame(result, columns = vocab)
    okt_tf_table['label'] = label

elif k == 2:
    vocab = kkma_list_final

    for i in range(N): # 각 문서에 대해서 아래 명령을 수행
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]        
            result[-1].append(tf(t, d))

    kkma_tf_table = pd.DataFrame(result, columns = vocab)
    kkma_tf_table['label'] = label

elif k == 3:
    vocab = komoran_list_final

    for i in range(N): # 각 문서에 대해서 아래 명령을 수행
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]        
            result[-1].append(tf(t, d))

    komoran_tf_table = pd.DataFrame(result, columns = vocab)
    komoran_tf_table['label'] = label

elif k == 4:
    vocab = mecab_list_final

    for i in range(N): # 각 문서에 대해서 아래 명령을 수행
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]        
            result[-1].append(tf(t, d))

    mecab_tf_table = pd.DataFrame(result, columns = vocab)
    mecab_tf_table['label'] = label


In [ ]:
# #wordcloud 저장
# counts = Counter(pos_list3)
# tags = dict(counts.most_common(50))
# wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', min_font_size = 20, max_font_size = 200, 
#                       background_color='black',colormap = "winter", prefer_horizontal = 1.0, width=800, height=600).generate_from_frequencies(tags) 
# plt.figure(figsize=(10,10))
# plt.imshow(wordcloud) 
# plt.axis('off') 
# plt.savefig('hannanum.jpg', dpi = 300, bbox_inches='tight')
# plt.savefig('hannanum.eps', format = 'eps')

In [ ]:
#pdf 저장
# flist = glob.glob('hannanum.jpg')

# pgsize = A4

# c = canvas.Canvas('hannanum.pdf', pagesize = pgsize)

# w2 = pgsize[0]
# for fpath in flist:
#     im = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
#     w, h =im.shape
#     r = w/h
#     h2 = w2/r
#     c.drawImage(fpath, 0, 0, w2, h2)
#     c.showPage()
# c.save()